In [1]:
%config IPCompleter.greedy=True

In [2]:
import requests
import json
import os
import logging
from tqdm.notebook import tqdm
from datetime import datetime

In [3]:
# Allow logging
logging.basicConfig(level=logging.INFO)
# Get current path
ROOT_PATH = os.path.dirname(os.path.realpath('__file__'))

In [4]:
# Load TMDB api key
with open(os.path.join(ROOT_PATH, 'TMDBApi.json')) as json_file:
    data = json.load(json_file)
    apiKey = data['apiKey']

In [5]:
params = { "api_key": apiKey, "page": 1 }

In [6]:
r = requests.get('https://api.themoviedb.org/3/person/popular', params=params )

In [7]:
def save_image_actor(id_actor):
    base_url_image = "https://image.tmdb.org/t/p/w500{0}"
    directory_name = id_actor
    path_directory = os.path.join(ROOT_PATH, 'images', 'train', directory_name)        
    if not os.path.exists(path_directory):
        os.mkdir(path_directory)
    count = 0
    r = requests.get('https://api.themoviedb.org/3/person/{0}/images'.format(id_actor), params=params)
    data = r.json()
    if 'profiles' in data:        
        for profile in data['profiles']:
            count += 1
            url_image = base_url_image.format(profile["file_path"])
            page = requests.get(url_image)
            with open(os.path.join(path_directory, '{0}.jpg'.format(count)), 'wb') as file:
                file.write(page.content)
    pbar.update(1)

def save_id_database(data_per_page, filename):
    actors_ids = []
    for actor in data_per_page["results"]:
        actor_id = str(actor["id"])
        actors_ids.append(actor_id)
    type_write = 'w'
    if os.path.isfile(filename):
        type_write = 'a'
    actors_ids = '\n'.join(actors_ids)
    with open(filename, type_write) as file:
        file.write(actors_ids)

def read_database_ids(filename):
    actors_ids = []
    with open(filename) as f:
        actors_ids = [line.rstrip() for line in f]
    return actors_ids

In [8]:
def retrieve_actors_ids():
    params = { "api_key": apiKey, "page": 1 }
    r = requests.get('https://api.themoviedb.org/3/person/popular', params=params )
    data = r.json()
    total_pages = data['total_pages']
    with tqdm(total= total_pages) as pbar:
        str_date_now = datetime.today().strftime('%d-%m-%Y')
        path_filename_actors_ids = os.path.join(ROOT_PATH, 'database')
        temp_filename = os.path.join(path_filename_actors_ids, 'temp_database_{0}'.format(str_date_now))
        filename = os.path.join(path_filename_actors_ids, 'database_{0}'.format(str_date_now))
        for i in range(1, total_pages + 1):
            params['page'] = i
            r = requests.get('https://api.themoviedb.org/3/person/popular', params=params )
            data_per_page = r.json()
            save_id_database(data_per_page, temp_filename)
            pbar.update(1)

        actors_ids = read_database_ids(temp_filename)
        actors_ids.insert(0, str(len(actors_ids)))
        actors_ids='\n'.join(actors_ids)
        with open(filename, 'w') as file:
            file.write(actors_ids)
        os.remove(temp_filename)

In [9]:
retrieve_actors_ids()
actors_ids = read_database_ids(os.path.join(ROOT_PATH, 'database', 'database_13-06-2020'))
total_actors = int(actors_ids[0])
with tqdm(total= total_actors) as pbar:    
    for actor_id in actors_ids[1:]:
        save_image_actor(actor_id)